In [11]:
import sys
import os
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torcheval.metrics import MulticlassAccuracy, MulticlassF1Score, MulticlassRecall, MulticlassPrecision, MulticlassConfusionMatrix

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            3, 6, 5
        )  # (3, 256, 256) -> new dim (256 - 5 + (2 * 0)) / 1 + 1 ) = 252 (6, 252, 252)
        self.pool = nn.MaxPool2d(2, 2)  # (6, 252, 252) -> (6, 126, 126)
        self.conv2 = nn.Conv2d(
            6, 16, 5
        )  # (6, 126, 126) -> (16, 122, 122) -> MaxPool -> (16, 61, 61)
        # self.fc1 = nn.Linear(16 * 61 * 61, 2048)
        self.fc1 = nn.Linear(16 * 13 * 13, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:

def create_dataloaders(dataset):
    # data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    data_loader = DataLoader(dataset, shuffle=True)
    dataiter = iter(data_loader)
    images, labels = next(dataiter)
    # print(images, labels)
    print(dataset.classes)
    # imshow(torchvision.utils.make_grid(images))
    labels = np.array([dataset.targets[i] for i in range(len(dataset))])

    # Define split sizes
    train_size = 0.7  # 70% for training
    val_size = 0.15  # 15% for validation
    test_size = 0.15  # 15% for testing

    # First, split into train + temp (val + test)
    train_idx, temp_idx, _, temp_labels = train_test_split(
        np.arange(len(dataset)),
        labels,
        stratify=labels,
        test_size=(1 - train_size),
        random_state=42,
    )

    # Then, split temp into validation and test
    val_idx, test_idx = train_test_split(
        temp_idx,
        stratify=temp_labels,
        test_size=(test_size / (test_size + val_size)),
        random_state=42,
    )

    # Create samplers
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # Create DataLoaders
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    # test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler)
    test_loader = DataLoader(dataset, sampler=test_sampler)
    return train_loader, val_loader, test_loader

def load_dataset(directory_path):
    transform = transforms.Compose(
        [transforms.Resize((64,64)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    dataset = torchvision.datasets.ImageFolder(root=directory_path, transform=transform)
    return dataset

In [14]:
dataset = load_dataset('~/sgoinfre/leaves/images/Apple')
train_loader, val_loader, test_loader = create_dataloaders(dataset)


['Apple_Black_rot', 'Apple_healthy', 'Apple_rust', 'Apple_scab']


In [15]:
def compute_validation_metrics(net, validation_loader):
    net.eval()
    criterion = nn.CrossEntropyLoss()

    validation_metrics = {'loss': 0}
    with torch.no_grad():
        f1_score = MulticlassF1Score(num_classes=4, average="macro")
        for inputs, labels in validation_loader:
            outputs = net(inputs)
            preds = torch.argmax(outputs, dim=1)
            
            validation_metrics["loss"] += criterion(outputs, labels)
            f1_score.update(preds, labels)
    validation_metrics["loss"] /= len(validation_loader)
    validation_metrics["f1_score"] = f1_score.compute()
    net.train()
    return validation_metrics

def update_validation_metrics_history(net, validation_loader, validation_metrics_history):
    new_validation_metrics = compute_validation_metrics(net, validation_loader)
    for name, value in new_validation_metrics.items():
        validation_metrics_history[name].append(value)

In [18]:
def train_model(train_loader, validation_loader):
    net = CNN()
    print(net)
    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.AdamW(net.parameters(), lr=0.001)
    validation_metrics_history = {
        "f1_score": [],
        "loss": [],
        "accuracy": []
    }
    for epoch in range(100):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            print(i)
        print(validation_metrics_history)
        update_validation_metrics_history(net, validation_loader, validation_metrics_history)
        print(validation_metrics_history)
        print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
    return net


In [19]:
net = train_model(train_loader, val_loader)


CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2704, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
{'f1_score': [], 'loss': [], 'accuracy': []}
{'f1_score': [tensor(0.5727)], 'loss': [tensor(0.5967)], 'accuracy': []}
[1,    35] loss: 0.017
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
{'f1_score': [tensor(0.5727)], 'loss': [tensor(0.5967)], 'accuracy': []}
{'f1_score': [tensor(0.5727), tensor(0.8406)], 'loss': [tensor(0.5967), tensor(0.3684)], 'accuracy': []}
[2,    35] loss: 0.009
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26

In [20]:
def test_model(net, test_loader, num_classes):
    net.eval()

    metrics = {
        "accuracy": MulticlassAccuracy(),
        "recall": MulticlassRecall(num_classes=num_classes, average="macro"),
        "precision": MulticlassPrecision(num_classes=num_classes, average="macro"),
        "f1_score": MulticlassF1Score(num_classes=num_classes, average="macro"),
        "confusion_matrix": MulticlassConfusionMatrix(num_classes=num_classes)
    }

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = net(images)  
            preds = torch.argmax(outputs, dim=1)  

            for metric in metrics.values():
                metric.update(preds, labels)

    results = {name: metric.compute() for name, metric in metrics.items()}

    for name, value in results.items():
        print(f"Test {name.capitalize()}: {value}")
    

In [21]:
test_model(net, test_loader, len(dataset.classes))

Test Accuracy: 0.918067216873169
Test Recall: 0.9191710948944092
Test Precision: 0.8940721154212952
Test F1_score: 0.9055185317993164
Test Confusion_matrix: tensor([[ 87.,   1.,   1.,   4.],
        [  4., 231.,   3.,   9.],
        [  0.,   0.,  41.,   1.],
        [  4.,   9.,   3.,  78.]])
